# Tools for NLP 

In [6]:
import findspark
findspark.init('/home/ubuntu/Spark/spark-3.3.0-bin-hadoop3')

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/23 12:01:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/23 12:01:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [42]:
sentenceData = spark.createDataFrame([
    (0.0, 'Hi I heard about Spark'),
    (0.0, 'I wish Java could use case classes'),
    (1.0, 'Logistic,Regression,models,are,neat'),
    
], ['label', 'sentence'])

In [43]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java could...|
|  1.0|Logistic,Regressi...|
+-----+--------------------+



In [44]:
tokenzier = Tokenizer(inputCol='sentence', outputCol='words')

In [45]:
words_data = tokenzier.transform(sentenceData)

In [46]:
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  0.0|I wish Java could...|[i, wish, java, c...|
|  1.0|Logistic,Regressi...|[logistic,regress...|
+-----+--------------------+--------------------+



In [47]:
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic,Regression,models,are,neat|[logistic,regression,models,are,neat]     |
+-----+-----------------------------------+------------------------------------------+



In [48]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [49]:
featurize_Data = hashing_tf.transform(words_data)

In [50]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [51]:
idf_model = idf.fit(featurize_Data)

In [52]:
rescaled_data = idf_model.transform(featurize_Data)

In [53]:
rescaled_data.select('label', 'features').show(truncate=False)


22/09/23 12:13:55 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
22/09/23 12:13:56 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                   |
|0.0  |(262144,[19036,20719,5555

In [54]:
from pyspark.ml.feature import CountVectorizer

In [55]:
df = spark.createDataFrame([
    (0, 'a b c'.split(' ')),
    (1, 'a b b c a'.split(' ')),
    
], ['id', 'words'])

In [56]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [57]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [59]:
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2.0)

In [60]:
model = cv.fit(df)

In [61]:
result = model.transform(df)

In [62]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

